In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

In [4]:
df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [ ]:
df[State].

In [5]:
df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [11]:
columns_of_interest=['ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 'County', 'State',
       'Zipcode', 'Country','Visibility(mi)', 'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight']
condition=df['State'] =='TX'
txDF=df[condition][columns_of_interest]

txDF.head()

txDF['City'].value_counts()

Houston              93245
Austin               58553
Dallas               57823
San Antonio          21613
El Paso               9352
Fort Worth            8746
Arlington             3011
Irving                2697
Mesquite              2180
Grand Prairie         2159
Pflugerville          2152
Manor                 1941
Del Valle             1451
Spring                1398
Humble                1274
Plano                 1222
Richardson            1138
Carrollton            1089
Garland                989
Wichita Falls          819
Pasadena               718
Lewisville             691
Kingwood               688
Round Rock             671
Denton                 603
Hurst                  576
Lancaster              573
Duncanville            532
Grapevine              526
Bellaire               492
                     ...  
Newcastle                1
Northeast Travis         1
Carmine                  1
Knott                    1
Shiner                   1
Mc Camey                 1
C